# 데이터레이어 적용

<br>

### 제품 조회 (`view_item`)

<br>

#### 00. 이벤트 세팅
1. **변수 : `currency` - 상수 - 'KRW'**
- 전자상거래 금액 단위

<img src='image/GTM/Cap 2025-10-30 20-21-52.png' width=600>

<br>

2. **변수: [CJS] user_id $\rightarrow$ 맞춤 자바스크립트 : `user_id`** 
- 이벤트를 발생시키는 사용자의 웹 사이트 내 로그인 ID, (미 로그인시 `null`)를 반환하는 함수

```js
function() {
  var id_ = MEMBER_HASH
  if (id_ === '') {
    return null
  } else {
    return MEMBER_HASH
  }
}
```

<br>

3. **변수: Event Setting $\rightarrow$ Google 태그: 이벤트 설정**
- **[CJS] user_id**를 기반으로 GA의 모든 이벤트에 `user_id`를 매개변수로 포함하도록 설정

<img src='image/GTM/Cap 2025-10-30 21-04-35.png' width=600>

<br>

4. **태그: GA4 측정 ID $\rightarrow$ Google 태그**
- `Event Setting`이 GA로 전송되게 구성 매개변수 설정

<img src='image/GTM/Cap 2025-10-30 21-06-24.png' width=600>


<br>


<br>

#### 01. 제품 페이지 트리거 생성
- **[TG] Detail Page - View**

<img src='image/GTM/Cap 2025-10-30 23-21-06.png' width=600>

<br>



#### 02. 데이터레이어 스크립트 작성

<br>

- 데이터레이어 스크립트 작성 시 자주 사용하는 `document` 함수

| 분류           | 함수 / 속성                                  | 설명                              | 예시                                            |
| ------------ | ---------------------------------------- | ------------------------------- | --------------------------------------------- |
|  **요소 선택** | `document.getElementById(id)`            | id로 단일 요소 선택                    | `document.getElementById('product')`          |
|              | `document.getElementsByClassName(class)` | 클래스명으로 여러 요소 선택(HTMLCollection) | `document.getElementsByClassName('price')[0]` |
|              | `document.getElementsByTagName(tag)`     | 태그명으로 선택                        | `document.getElementsByTagName('img')[0]`     |
|              | `document.querySelector(selector)`       | CSS 선택자로 첫 번째 요소 선택             | `document.querySelector('.product .price')`   |
|              | `document.querySelectorAll(selector)`    | CSS 선택자로 모든 요소 선택(NodeList)     | `document.querySelectorAll('.cart-item')`     |
|  **콘텐츠 추출** | `.textContent`        | 요소의 텍스트 내용 반환       | `document.querySelector('.title').textContent`               |
|               | `.innerText`          | 화면에 보이는 텍스트(스타일 반영) | `document.querySelector('.title').innerText`                 |
|               | `.innerHTML`          | 내부 HTML 전체          | `document.querySelector('.desc').innerHTML`                  |
|               | `.value`              | `<input>` 등의 값      | `document.querySelector('#search').value`                    |
|               | `.dataset`            | `data-*` 속성값 접근     | `document.querySelector('.product').dataset.id`              |
|               | `.getAttribute(attr)` | 특정 속성값 읽기           | `el.getAttribute('href')` or `el.getAttribute('data-price')` |
| **문서 정보** | `document.title`             | 페이지 `<title>` 값                                 | `"상품 상세페이지"`                                    |
|              | `document.URL`               | 전체 URL                                          | `"https://site.com/?idx=123"`                   |
|              | `document.location.pathname` | 도메인 제외한 경로                                      | `"/product/123"`                                |
|              | `document.referrer`          | 이전 페이지 URL                                      | `"https://google.com"`                          |
|              | `document.readyState`        | 현재 로드 상태 (`loading`, `interactive`, `complete`) | `if (document.readyState === 'complete') {...}` |
| **이벤트 관련** | `document.addEventListener(event, callback)` | 이벤트 리스너 등록 | `document.addEventListener('DOMContentLoaded', fn)` |
|              | `window.addEventListener(event, callback)`   | 전역 이벤트     | `window.addEventListener('load', fn)`               |
|              | `element.addEventListener(event, callback)`  | 특정 요소 클릭 등 | `btn.addEventListener('click', fn)`                 |
| **탐색 / 반복** | `.children`, `.parentElement`, `.closest()` | DOM 구조 탐색        | `el.closest('.container')`                                                    |
|                | `.querySelectorAll()` + `forEach()`         | 여러 요소 순회         | `document.querySelectorAll('.item').forEach(i => console.log(i.textContent))` |
|                | `Array.from(NodeList)`                      | NodeList를 배열로 변환 | `Array.from(document.querySelectorAll('.price'))`                             |


<br>

- 제품 페이지의 HTML 요소를 탐색

```html
...

<script type="application/ld+json">{
    "@context": "http://schema.org",
    "@type": "Product",
    "name": "High content steam cream",
    "image": [
        "https://cdn.imweb.me/thumbnail/20180315/5aa94fc3a82c2.png",
        "https://cdn.imweb.me/thumbnail/20180315/5aa94fc3f201c.png"
    ],
    "description": "Vitrine products are guaranteed to benefit from the highest quality products and the most trusted brands aroun... ",
    "offers": {
        "@type": "Offer",
        "price": 27000,
        "priceCurrency": "KRW",
        "url": "https://chslee96-make-test.imweb.me/shop_view/?idx=178",
        "availability": "http://schema.org/InStock"
    }
}</script>

... 
```


<br>

- 맞춤 자바스크립트 작성 - **[CJS] Items (Detail Page)**
  - **제품 상세 페이지에서의 제품 정보를 반환하는 함수**

```js
function() {
  var items = [];
  var scripts = document.querySelectorAll('script[type="application/ld+json"]');
  var script_product = null;

  for (var i = 0; i < scripts.length; i++) {
    try {
      var json_ = JSON.parse(scripts[i].textContent.trim());
      if (json_['@type'] === 'Product') {
        script_product = json_;
        break;
      }
    } catch (e) {
      continue;
    }
  }

  if (!script_product || !script_product.offers) {
    return items;
  }

  var url = script_product.offers.url || '';
  var match = url.match(/[?&]idx=(\d+)(?:$|[&#?])/);
  var item_id = match ? match[1] : null;

  var parts = url.split('/');
  var item_category = parts.length > 3 ? parts[3] : '';
  var item_price = script_product.offers.price || null;

  items.push({
    item_id: item_id,
    item_name: script_product.name || '',
    item_category: item_category,
    price: item_price
  });

  return items;
}

```

- 데이터레이어 스크립트 작성 - **맞춤 HTML 태그: [CH] Detail Page - View**
  - 트리거 **[TG] Detail Page - View**
  - **[CJS] Items (Detail Page)**, **[Var] currency** 활용

  **$\rightarrow$ 데이터레이어가 'view_item'을 감지**

```html

<script>
  var items = {{[CJS] Items (Detail Page)}};

  window.dataLayer = window.dataLayer || []; // 데이터레이어 초기화
  window.dataLayer.push(
    {
      'event': 'view_item',
      'ecommerce': {
        'items': items,
        'currency' : {{[Var] currency}}
      }
    }
  );
</script>
```

<br>


<br>

#### 03. 데이터 레이어 변수 생성
- '데이터 영역 변수' - **[DLV] ecommerce.items**

<img src='image/GTM/Cap 2025-10-31 00-14-09.png' width=600>

<br>

#### 04. 데이터레이어 이벤트 감지 트리거 생성
- ** [CH] Detail Page - View** 를 통하여 데이터레이어가 `view_item` 이벤트를 감지

    $\rightarrow$ 이를 트리거 삼아 GA로 `view_item`이벤트 전송

    $\rightarrow$ **맞춤 이벤트: [CE] View Item**

<img src='image/GTM/Cap 2025-10-31 00-26-45.png' width=600>


<br>

#### 05. 이벤트 태그 생성
- **[Event] View Item**
-  이벤트 매개변수로 설정하여 GA 이벤트 생성

<img src='image/GTM/Cap 2025-10-31 00-22-50.png' width=600>

<br>


<br>

<hr>

<br>

### 유저 정보 수집 

<br>